# Historical Prices With the OpenBB Platform

This notebook demonstrates some of the ways to approach loading historical price data using the OpenBB Platform.  The action is in the Equity module; but first, we need to initialize the notebook with the import statements block.

## Import Statements

In [1]:
from datetime import datetime, timedelta

import pandas as pd
from openbb import obb


## The Equity Module

Historical market prices typically come in the form of OHLC+V - open, high, low, close, volume.  There may be additional fields returned by a provider, but those are the expected columns.  Granularity and amount of historical data will vary by provider and subscription status.  Visit their websites to understand what your entitlements are.

### openbb.equity.price.historical()

- This endpoint has the most number of providers out of any function. At the time of writing, choices are:

['alpha_vantage', 'cboe', 'fmp', 'intrinio', 'polygon', 'tiingo', 'yfinance']

- Common parameters have been standardized across all souces, `start_date`, `end_date`, `interval`.

- The default interval will be `1d`.

- The depth of historical data and choices for granularity will vary by provider and subscription status.  Refer to the website and documentation of each source understand your specific entitlements.

- For demonstration purposes, we will use the `openbb-yfinance` data extension.

In [2]:
df_daily = obb.equity.price.historical(symbol = "spy", provider="yfinance")
df_daily.to_df().head(1)


,open,high,low,close,volume,dividends,stock splits,capital gains
date,,,,,,,,
2022-11-22,396.63,400.07,395.15,399.9,60429000,0.0,0.0,0.0


To load the entire history available from a source, pick a starting date well beyond what it might be. For example, `1900-01-01`

In [3]:
df_daily = obb.equity.price.historical(symbol = "spy", start_date = "1990-01-01", provider="yfinance").to_df()
df_daily.head(1)


,open,high,low,close,volume,dividends,stock splits,capital gains
date,,,,,,,,
1993-01-29,43.97,43.97,43.75,43.94,1003200,0.0,0.0,0.0


#### Intervals

The intervals are entered according to this pattern:

- `1m` = One Minute
- `1h` = One Hour
- `1d` = One Day
- `1W` = One Week
- `1M` = One Month

The date for monthly value is the first or last, depending on the provider.  This can be easily resampled from daily data.

In [4]:
df_monthly = obb.equity.price.historical("spy", start_date="1990-01-01", interval="1M", provider="yfinance").to_df()
df_monthly.tail(2)


,open,high,low,close,volume,dividends,stock splits,capital gains
date,,,,,,,,
2023-10-01,426.62,438.14,409.21,418.20,1999149700,0.0,0.0,0.0
2023-11-01,419.20,456.38,418.65,455.02,1161239576,0.0,0.0,0.0


#### Resample a Time Series

`yfinance` returns the monthly data for the first day of each month.  Let's resample it to take from the last, using the daily information captured in the previous cells.

In [5]:
(
    df_daily[["open", "high", "low", "close", "volume"]]
    .resample("M")
    .agg(
        {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
    )
)


,open,high,low,close,volume
date,,,,,
1993-01-31,43.97,43.97,43.75,43.94,1003200
1993-02-28,43.97,45.12,42.81,44.41,5417600
1993-03-31,44.56,45.84,44.22,45.19,3019200
1993-04-30,45.25,45.25,43.28,44.03,2697200
1993-05-31,44.09,45.66,43.84,45.22,1808000
...,...,...,...,...,...
2023-07-31,442.92,459.44,437.06,457.79,1374632400
2023-08-31,456.27,457.25,433.01,450.35,1754764700
2023-09-30,453.17,453.67,422.29,427.48,1588673200


The block below packs an object with most intervals.

In [6]:
class HistoricalPrices:
    def __init__(self, symbol, start_date, end_date, provider, **kwargs) -> None:
        self.one: pd.DataFrame = obb.equity.price.historical(
            symbol=symbol, start_date=start_date, end_date=end_date, interval = "1m", provider=provider, **kwargs
        ).to_df().convert_dtypes()
        self.five: pd.DataFrame = obb.equity.price.historical(
            symbol=symbol, start_date=start_date, end_date=end_date, interval = "5m", provider=provider, **kwargs
        ).to_df().convert_dtypes()
        self.fifteen: pd.DataFrame = obb.equity.price.historical(
            symbol=symbol, start_date=start_date, end_date=end_date, interval = "15m", provider=provider, **kwargs
        ).to_df().convert_dtypes()
        self.thirty: pd.DataFrame = obb.equity.price.historical(
            symbol=symbol, start_date=start_date, end_date=end_date, interval = "30m", provider=provider, **kwargs
        ).to_df().convert_dtypes()
        self.sixty: pd.DataFrame = obb.equity.price.historical(
            symbol=symbol, start_date=start_date, end_date=end_date, interval = "60m", provider=provider, **kwargs
        ).to_df().convert_dtypes()
        self.daily: pd.DataFrame = obb.equity.price.historical(
            symbol=symbol, start_date=start_date, end_date=end_date, interval = "1d", provider=provider, **kwargs
        ).to_df().convert_dtypes()
        self.weekly: pd.DataFrame = obb.equity.price.historical(
            symbol=symbol, start_date=start_date, end_date=end_date, interval = "1W", provider=provider, **kwargs
        ).to_df().convert_dtypes()
        self.monthly: pd.DataFrame = obb.equity.price.historical(
            symbol=symbol, start_date=start_date, end_date=end_date, interval = "1M", provider=provider, **kwargs
        ).to_df().convert_dtypes()

def load_historical(
    symbol: str = "",
    start_date = None,
    end_date = None,
    provider = None,
    **kwargs
) -> HistoricalPrices:

    if symbol == "":
        display("Please enter a ticker symbol")
    if provider is None:
        provider = "yfinance"
    prices = HistoricalPrices(symbol, start_date, end_date, provider, **kwargs)

    return prices

prices = load_historical("spy")
display(prices.__dict__.keys())
prices.weekly.tail(2)


dict_keys(['one', 'five', 'fifteen', 'thirty', 'sixty', 'daily', 'weekly', 'monthly'])

,open,high,low,close,volume,dividends,stock splits,capital gains
date,,,,,,,,
2023-11-13,439.23,451.42,438.42,450.79,376539600,0.0,0,0
2023-11-20,450.53,456.38,450.52,455.02,233751276,0.0,0,0


To demonstrate the difference between sources, let's compare values for daily volume from several sources.

In [7]:
# Collect the data

yahoo = obb.equity.price.historical("spy", provider="yfinance").to_df()
alphavantage = obb.equity.price.historical("spy", provider = "alpha_vantage").to_df()
intrinio = obb.equity.price.historical("spy", provider="intrinio").to_df()
fmp = obb.equity.price.historical("spy", provider="fmp").to_df()

# Make a new DataFrame with just the volume columns
compare = pd.DataFrame()
compare["AV Volume"] = alphavantage["volume"].tail(10)
compare["FMP Volume"] = fmp["volume"].tail(10)
compare["Intrinio Volume"] = intrinio["volume"].tail(10)
compare["Yahoo Volume"] = yahoo["volume"].tail(10)

compare


,AV Volume,FMP Volume,Intrinio Volume,Yahoo Volume
date,,,,
2023-11-09,83174417.0,83071417,83174417.0,83174400
2023-11-10,89558054.0,89558054,89558054.0,89462200
2023-11-13,52236068.0,52192568,52236068.0,52236100
2023-11-14,97176935.0,97130503,97176935.0,97176900
2023-11-15,77327573.0,77327573,77327573.0,77327600
2023-11-16,66665797.0,66654468,66665797.0,66665800
2023-11-17,83193902.0,83193902,83193902.0,83133200
2023-11-20,70055633.0,69614633,70055633.0,69936200
2023-11-21,49244639.0,49244639,49244639.0,49244600


## Other Types of Symbols

Other types of assets and ticker symbols can be loaded from `obb.equity.price.historical()`, below are some examples but not an exhaustive list.

### Share Classes

Some sources use `-` as the distinction between a share class, e.g., `BRK-A` and `BRK-B`. Other formats include:

- A period: `BRK.A`
- A slash: `BRK/A`
- No separator, the share class becomes the fourth or fifth letter.

```python
obb.equity.price.historical("brk.b", provider="polygon")
```

```python
obb.equity.price.historical("brk-b", provider="fmp")
```

While some providers handle the different formats on their end, others do not. This is something to consider when no results are returned from one source. Some may even use a combination, or accept multiple variations. Sometimes there is no real logic behind the additional characters, `GOOGL` vs. `GOOG`. These are known unknown variables of ticker symbology, what's good for one source may return errors from another. 

### Regional Identifiers

With providers supporting market data from multiple jurisdictions, the most common method for requesting data outside of US-listings is to append a suffix to the ticker symbol (e.g., `RELIANCE.NS` for Indian equities). Formats may be unique to a provider, so it is best to review the source's documentation for an overview of their specific conventions. [This page](https://help.yahoo.com/kb/SLN2310.html) on Yahoo describes how they format symbols, which many others follow to some degree.

### Indexes

Sources will have their own treatment of these symbols, some examples are:

- YahooFinance/FMP/CBOE: ^RUT
- Polygon: I:NDX

### Currencies

FX symbols face the same dilemna as share classes, there are several variations of the same symbol.

- YahooFinance: `EURUSD=X`
- Polygon: `C:EURUSD`
- AlphaVantage/FMP: `EURUSD`

**The symbol prefixes are handled internally when `obb.currency.price.historical()` is used to enter a pair with no extra characters.**

### Crypto

Similar, but different to FX tickers.

- YahooFinance: `BTC-USD`
- Polygon: `X:BTCUSD`
- AlphaVantage/FMP: `BTCUSD`

**The symbol prefixes are handled internally when `obb.crypto.price.historical()` is used to enter a pair with no extra characters and placing the fiat currency second.**

### Futures

Historical prices for active contracts, and the continuation chart, can be fetched via `yfinance`.

- Continuous front-month: `CL=F`
- December 2023 contract: `CLZ24.NYM`
- March 2024 contract: `CLH24.NYM`

Individual contracts will require knowing which of the CME venues the future is listed on. `["NYM", "NYB", "CME", "CBT"]`.

### Options

Individual options contracts are also loadable from `openbb.equity.price.historical()`.

- YahooFinance: `SPY241220P00400000`
- Polygon: `O:SPY241220P00400000`

These examples represent only a few methods for fetching historical price data.  Explore the contents of each module to find more!

In [8]:
obb.equity.price.historical("SPY241220P00400000", provider="yfinance").to_df()


,open,high,low,close,volume,dividends,stock splits
date,,,,,,,
2022-11-22,43.08,44.24,43.00,43.00,287,0.0,0.0
2022-11-23,41.58,42.45,39.00,39.00,6,0.0,0.0
2022-11-28,42.50,42.90,42.50,42.90,18,0.0,0.0
2022-11-30,41.00,41.00,41.00,41.00,1,0.0,0.0
2022-12-01,40.73,40.81,39.85,40.25,42,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-16,11.41,11.88,11.27,11.70,264,0.0,0.0
2023-11-17,11.50,11.50,11.19,11.40,0,0.0,0.0
2023-11-20,11.00,11.10,10.50,10.76,121,0.0,0.0


In [9]:
obb.equity.price.historical("^RUT", provider="cboe").to_df()


,open,high,low,close,volume
date,,,,,
2020-11-27,1846.26,1855.27,1845.50,1855.27,0
2020-11-30,1854.87,1854.87,1813.56,1819.82,0
2020-12-01,1822.92,1848.02,1822.92,1836.05,0
2020-12-02,1833.08,1842.53,1817.02,1838.03,0
2020-12-03,1838.52,1860.35,1838.52,1848.70,0
...,...,...,...,...,...
2023-11-16,1797.00,1797.00,1767.42,1773.76,0
2023-11-17,1784.86,1798.44,1784.86,1797.77,0
2023-11-20,1798.56,1809.21,1794.71,1807.08,0


In [10]:
obb.equity.price.historical("^RUT", provider="fmp").to_df()


,open,high,low,close,volume,vwap,label,adj_close,unadjusted_volume,change,change_percent,change_over_time
date,,,,,,,,,,,,
2022-11-22,1843.869995,1861.079956,1840.040039,1860.439941,3887990000,1853.85,"November 22, 22",1860.439941,3.887990e+09,16.56995,0.89865,0.008986
2022-11-23,1857.589966,1868.920044,1851.680054,1863.520020,3279720000,1861.37,"November 23, 22",1863.520020,3.279720e+09,5.93005,0.31923,0.003192
2022-11-25,1862.920044,1873.589966,1861.000000,1869.189941,1706460000,1867.93,"November 25, 22",1869.189941,1.706460e+09,6.26990,0.33656,0.003366
2022-11-28,1857.180054,1857.520020,1827.079956,1830.959961,3615430000,1838.52,"November 28, 22",1830.959961,3.615430e+09,-26.22009,-1.41000,-0.014100
2022-11-29,1832.520020,1846.069946,1832.520020,1836.550049,3546040000,1838.38,"November 29, 22",1836.550049,3.546040e+09,4.03003,0.21992,0.002199
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-16,1797.000000,1797.000000,1767.420040,1773.760010,3964520000,1783.80,"November 16, 23",1773.760010,3.964520e+09,-23.23999,-1.29000,-0.012900
2023-11-17,1784.859990,1798.439940,1784.859990,1797.770020,3777240000,1791.48,"November 17, 23",1797.770020,3.777240e+09,12.91003,0.72331,0.007233
2023-11-20,1798.560060,1809.209960,1794.709960,1807.079960,3644790000,1802.39,"November 20, 23",1807.079960,3.644790e+09,8.51990,0.47371,0.004737


In [11]:
obb.equity.price.historical("CLZ24.NYM", provider="yfinance").to_df()


,open,high,low,close,volume,dividends,stock splits
date,,,,,,,
2022-11-22,72.00,72.50,71.70,71.91,6101,0.0,0.0
2022-11-23,72.01,72.12,70.53,71.33,10513,0.0,0.0
2022-11-25,71.21,72.21,70.55,70.66,5609,0.0,0.0
2022-11-28,70.50,71.53,69.26,71.21,12409,0.0,0.0
2022-11-29,70.80,72.53,70.61,71.77,14418,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-16,73.94,74.10,70.94,71.34,58159,0.0,0.0
2023-11-17,71.37,73.69,71.21,73.58,36644,0.0,0.0
2023-11-20,73.05,75.39,73.05,74.98,38285,0.0,0.0


In [12]:
obb.equity.price.historical("CL=F", provider="fmp").to_df()


,open,high,low,close,volume,vwap,label,adj_close,unadjusted_volume,change,change_percent,change_over_time
date,,,,,,,,,,,,
2022-11-22,80.22,82.36,79.85,80.95,269615,81.05,"November 22, 22",80.949997,269615.0,0.73,0.91000,0.009100
2022-11-23,81.08,81.95,76.83,77.94,379145,78.91,"November 23, 22",77.940002,379145.0,-3.14,-3.87000,-0.038700
2022-11-25,77.51,79.90,76.22,76.28,266451,77.47,"November 25, 22",76.279999,266451.0,-1.23,-1.59000,-0.015900
2022-11-28,75.93,77.84,73.60,77.24,405033,76.23,"November 28, 22",77.239998,405033.0,1.31,1.73000,0.017300
2022-11-29,76.54,79.65,76.29,78.20,381869,78.05,"November 29, 22",78.199997,381869.0,1.66,2.17000,0.021700
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-17,72.97,75.99,72.75,75.89,101484,74.88,"November 17, 23",75.890000,101484.0,2.92,4.00000,0.040000
2023-11-19,75.65,75.80,75.65,75.80,10,75.75,"November 19, 23",75.800000,10.0,0.15,0.19828,0.001983
2023-11-20,75.65,78.22,75.65,77.60,270638,77.16,"November 20, 23",77.600000,270638.0,1.95,2.58000,0.025800
